# **Text, Web, & Media Analytics Assignment 2**

# Setup

In [1]:
# NOTE: ORDER ME PLEASE 🙇‍♂️

import nltk
import pandas as pd

import os
import regex as re
import string
import math
import csv

import sklearn
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Define the class for Bag-of-Word representation
class bow_document:
    def __init__(self, item_id: str):
        # Type check to ensure object is initialised correctly
        if not isinstance(item_id, str):
            raise TypeError("item_id: value must be a string.")
            # Technically could work with str or int indexing (for key in collection),
            # using *only* str ensures no double-up of pointers
            # (e.g. item_id '1' vs item_id 1)

        self.doc_id = item_id  # assigning doc_id from 'item_id'
        self.terms = {}  # dictionary for terms and their frequencies
        self._doc_len = 0  # document length, private attribute

    def add_term(self, term: str):
        """Add a term to the document or update its frequency if it already exists."""
        
        # Type check to ensure term is a str
        if not isinstance(term, str):
            raise TypeError("term: value must be a string.")
        
        self.doc_len += 1  # extend doc_len

        if term in self.terms:
            self.terms[term] += 1  # add frequency if the term exists
        else:
            self.terms[term] = 1  # if it doesn't exist, add it (setting frequency to 1)
        
    def get_doc_id(self) -> str:
        """Return the document ID."""
        return self.doc_id
    
    def get_term_list(self, sorted_by_freq: bool = None) -> dict:
        """
        Return a list of terms occurring in the document, optionally sorted by their frequency.
        If sorted_by_freq is True, the terms are returned sorted by their frequency in descending order.
        If sorted_by_freq is False or None (default), the terms are returned in arbitrary order.
        """

        # Type check to ensure sorted_by_freq is either None or a boolean
        if not isinstance(sorted_by_freq, (bool, type(None))):
            raise TypeError("sorted_by_freq: must be a boolean or None.")

        if sorted_by_freq:
            # If sorted_by_freq is True
            sorted_terms = sorted(self.terms.items(), key=lambda word: word[1], reverse=sorted_by_freq)  # generate a sorted list of terms by frequency
            return {term: freq for term, freq in sorted_terms}  # return key:value pairs based on sorted terms
        else:
            # If sorted_by_freq is False or None, return the terms as is (i.e., unsorted and as they are added in)
            return self.terms
        
    def get_bag_of_words(self, sorted_by_freq: bool = None) -> str:
        """Return full bag-of-words representation for bow_document object, including; doc_id, term_count, doc_len, and terms."""
        
        # Type check to ensure sorted_by_freq is either None or a boolean
        if not isinstance(sorted_by_freq, (bool, type(None))):
            raise TypeError("sorted_by_freq: must be a boolean or None.")

        # Defining formatted string for bag-of-word representation
        bag_of_words = f"""doc_id='{self.doc_id}',term_count={len(self.get_term_list())},doc_len={self.doc_len},terms={self.get_term_list(sorted_by_freq)}"""

        return bag_of_words  # return BOW representation; this kind of data can be stored and "unpacked" easily
    
    @property  # accessor (get) method for doc_len
    def doc_len(self) -> int:
        """The doc_len property getter method."""
        return self._doc_len

    @doc_len.setter  # mutator (setter) method for doc_len
    def doc_len(self, value: int):
        """The doc_len property setter method."""
        if not isinstance(value, int):
            raise TypeError("doc_len: must be an int.")
        if value < 0:
            raise ValueError("doc_len: must not be negative.")
        
        self._doc_len = value

# Define the class for collection of bow_document objects
class bow_document_collection:
    def __init__(self):
        self.docs = {}  # initialise dictionary to hold collection (dict) of doc_id:bow_document

        self.term_doc_count = {}  # initialise dictionary to track the number of documents each term appears in

    # Method to add a doc (bow_document object)
    def add_doc(self, doc: bow_document):
        """Add bow_document object to the collection, using doc_id as the key, and update the inverted index."""

        # Type check to ensure doc is a bow_document object
        if not isinstance(doc, bow_document):
            raise TypeError("doc: must be an instance of bow_document.")
        
        # Add to the docs dict; key as doc_id and value as bow_document object (doc_id:bow_document)
        self.docs[doc.get_doc_id()] = doc

        # Update term document count for each term
        for term in doc.terms:
            if term in self.term_doc_count:
                self.term_doc_count[term] += 1  # add one if the term exists in the corpus dictionary
            else:
                self.term_doc_count[term] = 1  # if it does not exist in the corpus dictionary, initialise by setting to 1
    
    def get_collection_ids(self) -> str:
        """Return list of document IDs present in the collection."""

        # Type check to ensure doc_id is a string
        if not len(self.docs) > 0:
            raise AttributeError("bow_document_collection object is empty, no IDs to return.")  # Corrected to match the check
        
        doc_ids_str = "'" + "', '".join(self.docs.keys()) + "'"  # create a string that lists doc_ids

        collection_ids = f"bow_document_collection(doc_ids: {doc_ids_str})"  # format the return variable

        return collection_ids

In [5]:
def stop_word_parser(stop_word_path: str) -> list:
    """Parse defined list of stop words (assumes txt file with words delimited with ',')."""

    # Type check to ensure stop_word_path is a str
    if not isinstance(stop_word_path, str):
        raise TypeError("stop_word_path: value must be a str.")
    
    # NOTE: need attribute check the path exists

    # Open file in read mode
    with open(stop_word_path, 'r') as file:
        stop_words = file.read()  # read text in given file into stop_words

    # We know what the format is ahead of time, so not a lot of processing needed;
    # i.e., assumes we don't need to make something more robust and that we're using the same txt
    stop_words = stop_words.lower().split(",")  # tokenize stop_words; delimited with ','
    stop_words = list(set(stop_words))  # reduce stop_words to uniques
    
    return stop_words  # return stop_words as a list object

def tokenization(words: str) -> list:
    """Tokenize input text by removing line breaks, numbers, punctuation, normalizing whitespace, stripping leading/trailing spaces, and splitting into lowercased words."""

    # Type check to ensure words is a str
    if not isinstance(words, str):
        raise TypeError("words: value must be a str.")

    words = words.replace("\n", "")  # don't want line breaks to contribute
    words = re.sub(r'\d', '', words)  # not interested in numbers for this particular task, remove
    words = re.sub(f'[{re.escape(string.punctuation)}]', ' ', words)  # not interested in punctuation, remove
    words = re.sub(r'\s+|\t+|\v+|\n+|\r+|\f+', ' ', words).strip()  # standardise the whitespaces, remove leading/trailing whitespace
    words = words.lower()  # standardise words as lower
    words = words.split()  # tokenize, deftault split on space

    # Filter out small words; can be important in some queries, usually in combinations, opting not to handle for simplicity.
    # For example, with no discrete management of apostrophes (indicating contractions or posession) aside from replacement 
    # of punctuation with a single space, we will get the following: "Amelia's" → ["Amelia", "s"] → ["Amelia"].
    # Unless they are actual words (e.g., "I" versus "s" or "t"), they won't be removed in stopping process.
    words = [word for word in words if len(word) >= 3]

    return words  # return list object of string words

def xml_parser(stop_words: list, xml_path: str) -> bow_document:
    """Parse a single XML file, process text, and return an bow_document object with term frequencies."""
    
    # Type check to ensure stop_words is a list of str
    if not isinstance(stop_words, list) or not all(isinstance(word, str) for word in stop_words):
        raise TypeError("stop_words: must be a list of strings.")
    
    # Type check to ensure xml_path is a str
    if not isinstance(xml_path, str):
        raise TypeError("xml_path: value must be a str.")
    
    # Check if provided xml_path is a valid xml file, raise AttributeError if it is not
    if not ((os.path.isfile(xml_path)) and (xml_path.lower().endswith(".xml"))):
        raise AttributeError(f"""xml_path: '{xml_path}' is not a valid xml file.""")
        # NOTE: check is included here for targeting single xml (wheras parse_rcv1v2() executes this check in loop)

    # DOCUMENT PARSING - recognition of the content and structure of text documents
    # Open file in read mode
    with open(xml_path, 'r') as file:
        xml = file.read()  # read xml in given file

    text = re.search(r'<text>\s*((?:<p>.*?</p>\s*)+)</text>', xml, re.DOTALL)  # find all text within the <text> tag

    # If no text found, raise attribute error; else return match group 1
    if not text:
        raise AttributeError(fr"""xml_path: '{xml_path}' did not contain any text, see text tag (expect match at '<text>\s*((?:<p>.*?</p>\s*)+)</text>' with re.DOTALL).""") 
    else:
        text = text.group(1)

    # Replace HTML entities with their corresponding characters
    html_entities = {"&lt;": "<", "&gt;": ">", "&amp;": "&", "&quot;": "\"", "&apos;": "'", "&nbsp;": " " }
    for entity, char in html_entities.items():
        text = text.replace(entity, char)
    
    text = re.sub(r'<.*?>', ' ', text).strip()  # remove any XML tags (p tags in our case)
    
    # TOKENIZING - forming words from sequence of characters; critically, generating a list of tokens
    words = tokenization(text)
    
    # POSTING - a collection of arbitrary data (including a pointer)
    item_id = re.search(r'<newsitem itemid="(\d+)"', xml)  # POINTER - a unique identifier of a document (item_id attribute from newsitem element in this case)

    if not item_id:
        # If no item_id found, raise attribute error
        raise AttributeError(f"""xml_path: '{xml_path}' did not contain pointer, see item_id attribute in newsitem tag (expect match at '<newsitem itemid="(\\d+)"').""") 
    else:
        item_id = item_id.group(1)  # otherwise, take group 1 of regex (just the \d+ match component)
        
    document = bow_document(item_id)  # initialise bow_document object with the pointer (item_id)

    # STOPPING - removing stop (function) words from the text being analysed; have little meaning on their own
    words = [word for word in words if word not in stop_words]
    
    # STEMMING - reducing words to their word stem, base or root form (remove morphological variations)
    stemmer = nltk.stem.PorterStemmer()  # Porter Stemmer: efficient for information retrieval and text processing tasks – can often create non-words in favour of faster speeds
    words = [stemmer.stem(word) for word in words] 
    
    # Iterate over each stemmed word
    for stemmed_word in words:
        document.add_term(stemmed_word)  # use method add_term to update the bow_document object (our arbitrary data)          

    return document  # return the bow_document object

def parse_rcv1v2(stop_words: list, input_path: str) -> bow_document_collection:
    """Parse XML documents in a directory, filter stop words, and return a collection of bow_document objects."""
    
    # Type check to ensure stop_words is a list of str
    if not isinstance(stop_words, list) or not all(isinstance(word, str) for word in stop_words):
        raise TypeError("stop_words: must be a list of strings.")
    
    # Type check to ensure input_path is a str
    if not isinstance(input_path, str):
        raise TypeError("input_path: value must be a str.")
    
    # NOTE: need to do attribute check to see if input_path exists

    collection = bow_document_collection()  # initialise bow_document_collection object (collection of bow_document objects)
    
    # Iterate through files in directory
    for xml_file in os.listdir(input_path):
        xml_path = os.path.join(input_path, xml_file)  # build path to files
        if ((os.path.isfile(xml_path)) and (xml_path.lower().endswith(".xml"))):
            doc = xml_parser(stop_words, xml_path)  # parse xml with xml_parser function
            collection.add_doc(doc)  # use method add_doc to update the bow_document_collection object

    # If no xmls parsed (i.e., collection length is 0), raise attribute error
    if len(collection.docs) == 0:
        raise AttributeError(f"""input_path: '{input_path}' did not contain any valid xml files.""")

    return collection  # return the bow_document_collection object

def parse_query(query: str, stop_words: list) -> dict:
    """Tokenize an input query, remove stop words, and return a dictionary of remaining word frequencies."""

    # Type check to ensure stop_words is a list of str
    if not isinstance(stop_words, list) or not all(isinstance(word, str) for word in stop_words):
        raise TypeError("stop_words: must be a list of strings.")
    
    # Type check to ensure query is a str
    if not isinstance(query, str):
        raise TypeError("query: value must be a string.")
    
    # TOKENIZING - forming words from sequence of characters; critically, generating a list of tokens
    words = tokenization(query)
    
    # STOPPING - removing stop (function) words from the text being analysed; have little meaning on their own
    words = [word for word in words if word not in stop_words]
    
    # STEMMING - reducing words to their word stem, base or root form (remove morphological variations)
    stemmer = nltk.stem.PorterStemmer()  # Porter Stemmer: efficient for information retrieval and text processing tasks – though can often create non-words in favour of faster speeds
    words = [stemmer.stem(word) for word in words]
    
    # Constrcut term:frequency dictionary by counting instances of each word (more efficient than for loop + if/else)
    query_term_frequency = {stemmed_word: words.count(stemmed_word) for stemmed_word in set(words)}

    return query_term_frequency  # return the dictionary containing word frequencies

In [6]:
def parse_queries(file_path: str) -> pd.DataFrame:
    # Type check to ensure the file_path is a string
    if not isinstance(file_path, str):
        raise TypeError("file_path: value must be a string.")
    
    with open(file_path, 'r') as file:
        data = file.read()
    
    # Define regex pattern to split queries
    query_pattern = re.compile(r'<Query>(.*?)</Query>', re.DOTALL)
    queries = query_pattern.findall(data)
    
    # Initialize lists for storing parsed data
    nums, titles, descriptions, narratives = [], [], [], []
    
    # Define regex patterns to extract individual fields
    num_pattern = re.compile(r'<num>\s*Number:\s*R(\w+)', re.MULTILINE)
    title_pattern = re.compile(r'<title>([\w\s,.-]*)', re.MULTILINE)
    desc_pattern = re.compile(r'<desc>\s*Description:\s*(.*?)\n\n', re.DOTALL)
    narr_pattern = re.compile(r'<narr>\s*Narrative:\s*(.*?)\n\n', re.DOTALL)
    
    for query in queries:
        # Extract data using regex patterns
        num_match = num_pattern.search(query)
        title_match = title_pattern.search(query)
        desc_match = desc_pattern.search(query)
        narr_match = narr_pattern.search(query)
        
        nums.append(num_match.group(1) if num_match else pd.NA)
        titles.append(title_match.group(1).strip() if title_match else pd.NA)
        descriptions.append(desc_match.group(1).strip() if desc_match else pd.NA)
        narratives.append(narr_match.group(1).strip() if narr_match else pd.NA)

    # Create a pandas DataFrame
    query_frame = pd.DataFrame({
        'Number': nums,
        'Title': titles,
        'Description': descriptions,
        'Narrative': narratives
    })
    
    return query_frame

def write_scores_to_file(scores: dict, filename: str):
    """
    Write the scores dictionary to a .dat file.
    """

    if not isinstance(scores, dict):
        raise TypeError("scores: value must be a dictionary.")
    
    if not all((isinstance(doc_id, str)) and (isinstance(score, (int, float))) for doc_id, score in scores.items()):
        raise ValueError("scores: dictionary must consist of string keys (for documents) and int/float values (for document scores).")

    if not isinstance(filename, str):
        raise TypeError("filename: value must be a string.")
    
    # Combine the directory and filename to form the full path
    directory = 'RankingOutputs'
    filepath = os.path.join(directory, filename)

    # Check if the directory exists, and create it if it doesn't
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(filepath, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')  # Using tab delimiter for .dat format
        for doc_id, score in scores.items():
            writer.writerow([doc_id, score])

# Task 1: BM25

**Description:** Design a BM25-based IR model (**BM25**) that ranks documents in each data collection using the corresponding topic (query) for all 50 data collections.


**Inputs:** 50 long queries (topics) in *the50Queries.txt* and the corresponding 50 data collections (*Data_C101, Data_C102, …, Data_C150*).


**Output:** 50 ranked document files (e.g., for Query *R107*, the output file name is “BM25_R107Ranking.dat”) for all 50 data collections and save them in the folder “RankingOutputs”.

For each long query (topic) $Q$, you need to use the following equation to calculate a score for each document $D$ in the corresponding data collection (dataset):

$\sum_{i \in Q} \log_{10}(\frac{(r_i + 0.5)/(R-r_i+0.5)}{(n_i-r_i+0.5)/(N-n_i-R+r_i+0.5)})\cdot\frac{(k_1+1)f_i}{K+f_i}\cdot\frac{(k_2+1)qf_i}{k_2+qf_i}$

- $Q$ is the title of the long query, 
- $k_1 = 1.2$
- $k_2=500$
- $b = 0.75$
- $dl = len(D)$
- $avdl$ is the average length of a document in the dataset. 
- $K = k1\cdot((1-b) + b\cdot dl /avdl)$
- The ***base of the log function is 10***. 

Note that *BM25 values can be negative*, and you may need to update the above equation to produce non-negative values but keep the resulting documents in the same rank order.

**Formally describe your design for BM25** in an algorithm to **rank documents in each data collection *using corresponding query* (topic) ***for all 50 data collections*****. When you use the BM25 score to rank the documents of each data collection, you also need to **answer what the query feature function and document feature function are**.

In [7]:
def BM25(collection: bow_document_collection, query: dict) -> dict:
    """
    BM25 ranking function for a collection of documents and a given query.
    Generates a score for a given documents term:frequency set.
    Incorporates term frequency (TF) and inverse document frequency (IDF) factors. 
    It accounts for term frequency saturation as well as document length bias.
    """
    
    # Type check to ensure coll is a bow_document_collection
    if not isinstance(collection, bow_document_collection):
        raise TypeError("collection: must be a bow_document_collection object.")
    
    # If no collection contains no documents, raise attribute error
    if len(collection.docs) == 0:
        raise AttributeError("bow_document_collectionection: object contains no documents (Rcv1Doc objects).")
    
    # Type check to ensure query is a dict
    if not isinstance(query, dict):
        raise TypeError("query: must be a dict object.")
    
    # Setting parameters
    k_1 = 1.2  # Controls non-linear term frequency normalization (saturation)
    k_2 = 500  # Controls non-linear term frequency normalization for query terms
    b = 0.75  # Controls to what degree document length normalizes tf values

    N = len(collection.docs)  # total number of documents in the collection
    R = 0  # number of relevant documents for this query; predefined by task
    r_i = 0  # number of relevant documents containing query term i; predefined by task

    # Calculate the average document length across the entire collection
    total_corpus_length = sum(doc.doc_len for doc in collection.docs.values())
    mean_doc_len = total_corpus_length / N
    
    doc_scores = {}  # initialize doc_score dictionary to store calculated scores

    # Loop through each term in the query.
    for query_term, query_frequency in query.items():
        n_i = collection.term_doc_count.get(query_term, 0)  # the number of documents containing term i (0 if not present)

        # Calculate the inverse document frequency for the term
        idf_component = math.log10(((r_i + 0.5)/(R - r_i + 0.5)) / ((n_i - r_i + 0.5) / (N - n_i - R + r_i + 0.5)))
        # idf_component = math.log10((N - n_i + 0.5) / (n_i + 0.5))  # NOTE: simplified, need feedback from Slack

        # Component measures the rarity of the term across the entire collection; 
        # term appearing in fewer documents will have a higher IDF, making it more influential.
        # Formula ensures that no division by zero occurs by introducing additive smoothing of 0.5 to the numerator and denominator.

        for doc_ID, doc in collection.docs.items():
            doc_len = doc.doc_len  # document length

            K = k_1 * ((1 - b) + b * doc_len / mean_doc_len)  # frequency normaliser
            
            document_term_frequency = doc.terms.get(query_term, 0)  # query term frequency within the document (0 if not present)
            
            # Calculate the term frequency normalization for the document term
            tf_component = ((k_1 + 1) * document_term_frequency) / (K + document_term_frequency)
            # This component adjusts the score based on the frequency of the term in the document.
            # The normalisation (denominator) prevents over-emphasis on terms that appear too frequently within a single document.
            # `k_1` controls the non-linear term frequency saturation, and `K` adjusts the weight based on document length.

            # Calculate the query term frequency normalization
            query_component = ((k_2 + 1) * query_frequency) / (k_2 + query_frequency)
            # Adjusts the score based on the query term's frequency.
            # Denominator prevents over-emphasis on query terms that appear frequently.
            
            score = idf_component * tf_component * query_component  # determine the score (can be non-negative, clamping used below to adjust)
            
            if doc_ID not in doc_scores:
                doc_scores[doc_ID] = 0  # initialize doc_score if not present
            
            doc_scores[doc_ID] += max(score, 0)  # update the document's score with the product of the IDF and TF components (clamping non-negatives to 0)
    
    doc_scores = dict(sorted(doc_scores.items(), key=lambda item: item[1], reverse=True))  # sort the results

    # Return the document score
    return doc_scores

In [8]:
stop_words = stop_word_parser('common-english-words.txt')

query_frame = parse_queries('the50Queries.txt')
query_frame['parsed_titles'] = query_frame['Title'].apply(lambda row: parse_query(row, stop_words))

In [6]:
document_set = {}

input_path = 'Data_Collection'
for collection_path in os.listdir(input_path):
    data_key = collection_path.split('_C', 1)[1]
    document_set[data_key] = parse_rcv1v2(stop_words, os.path.join(input_path, collection_path))

In [18]:
BM25_results = {}

for query_key, collection in document_set.items():
    query = query_frame.loc[query_frame['Number'] == query_key, 'parsed_titles'].iloc[0]
    BM25_results[query_key] = BM25(collection, query)
    write_scores_to_file(BM25_results[query_key], f"BM25_R{query_key}Ranking")

# Task 2: Jelinek-Mercer Language Model

**Description:** Design a Jelinek-Mercer based Language Model (**JM_LM**) that ranks documents in each data collection using the corresponding topic (query) for all 50 data collections.


**Inputs:** 50 long queries (topics) in *the50Queries.txt* and the corresponding 50 data collections (*Data_C101, Data_C102, …, Data_C150*).


**Output:** 50 ranked document files (e.g., for Query *R107*, the output file name is “JM_LM_R107Ranking.dat”) for all 50 data collections and save them in the folder RankingOutputs”.

For each long query (topic) $R_x$, you need to use the following equation to calculate a conditional probability for each document $D$ in the corresponding data collection (dataset):


$p(R_x|D)=\Pi_{i=1}^n ((1-\lambda)\cdot\frac{f_{q_i,D}}{|D|}+\lambda\cdot\frac{c_{q_i}}{|C|})$

- $f_{q_i,D}$ is the number of times query word $q_i$ occurs in document $D$
- $|D|$ is the number of word occurrences in $D$
- $c_{q_i}$ is the number of times query word $q_i$ occurs in the data collection $C$
- $|C|$ is the total number of word occurrences in data collection $C$
- `λ = 0.4`

**Formally describe your design for JM_LM** in an algorithm to **rank documents in each data collection *using corresponding query* (topic) ***for all 50 data collections*****. When you use the probabilities to rank the documents of each data collection, you also need to **answer what the query feature function and document feature function are**.

In [20]:
def JM_LM(collection, query):
    """
    Calculate the conditional probability of each document given a query using the Jelinek-Mercer smoothing Language Model.
    """
    
    # Type check to ensure collection is an bow_document_collection object
    if not isinstance(collection, bow_document_collection):
        raise TypeError("collection: must be a bow_document_collection object.")
    
    # Check if the collection contains any documents
    if len(collection.docs) == 0:
        raise AttributeError("bow_document_collection: object contains no documents (Rcv1Doc objects).")
    
    # Validate that the query is a dictionary
    if not isinstance(query, dict):
        raise TypeError("query: must be a dict object.")
    
    # Set lambda parameter for Jelinek-Mercer smoothing
    lambda_val = 0.4
    
    # Calculate the total length of the corpus by summing the lengths of all documents
    total_corpus_length = sum(doc.doc_len for doc in collection.docs.values())
    
    # Initialize an empty dictionary to store the scores for each document
    doc_scores = {}

    # Iterate through each term in the query
    for query_term in query:
        # Get the frequency of the query term in the entire collection
        c_qi = collection.term_doc_count.get(query_term, 0)
        
        # Iterate through each document in the collection
        for doc_ID, doc in collection.docs.items():
            # Get the frequency of the query term in the current document
            f_qi_D = doc.terms.get(query_term, 0)

            # Calculate the probability of the term occurring in the document
            p_doc = (f_qi_D / doc.doc_len) if doc.doc_len > 0 else 0
            
            # Calculate the probability of the term occurring in the whole collection
            p_coll = (c_qi / total_corpus_length) if total_corpus_length > 0 else 0

            # Calculate the smoothed score for the term using Jelinek-Mercer smoothing
            score = (1 - lambda_val) * p_doc + lambda_val * p_coll

            # Initialize the score for the document if not already done
            if doc_ID not in doc_scores:
                doc_scores[doc_ID] = 1  # Multiplicative identity

            # Multiply the score to the cumulative product if it's greater than zero
            if score > 0:
                doc_scores[doc_ID] *= score

    # Sort the documents by their score in descending order and return the sorted dictionary
    doc_scores = dict(sorted(doc_scores.items(), key=lambda item: item[1], reverse=True))

    # Return the document scores
    return doc_scores

In [21]:
JM_LM_results = {}

for query_key, collection in document_set.items():
    query = query_frame.loc[query_frame['Number'] == query_key, 'parsed_titles'].iloc[0]
    JM_LM_results[query_key] = JM_LM(collection, query)
    write_scores_to_file(JM_LM_results[query_key], f"JM_LM_R{query_key}Ranking")    

# Task 3: Pseudo-Relevance Model

**Description:** Based on the knowledge you gained from this unit, design a pseudo-relevance model (My_PRM) to rank documents in each data collection using the corresponding topic (query) for all 50 data collections.


**Inputs:** 50 long queries (topics) in the50Queries.txt and the corresponding 50 data collections (Data_C101, Data_C102, …, Data_C150).


**Output:** 50 ranked document files (e.g., for Query R107, the output file name is “My_PRM_R107Ranking.dat”) for all 50 data collections and save them in the folder RankingOutputs”.

**Formally describe your design for My_PRM** in an algorithm to **rank documents in each data collection *using corresponding query* (topic) ***for all 50 data collections*****.Your *approach should be generic*; that means it is feasible to be used for other topics (queries). You also need to **discuss the differences between My_PRM and the other two models (BM25 and JM_LM)**.

In [ ]:
def My_PRM():
    
    
    return

# Task 4: Model Testing

**Description:** Use Python to implement three models: `BM25`, `JM_LM`, and `My_PRM`, and **test them on the given 50 data collections for the corresponding 50 queries (topics)**. 

Design Python programs to implement these three models. You can use a .py file (or a .ipynb file) for each model.


For each long query, your python programs will produce ranked results and save them into .dat files. For example, for query R107, you can save the ranked results of three models into “BM25_R107Ranking.dat”, “JM_LM_R107Ranking.dat”, and “My_PRM_R107Ranking.dat”, respectively by using the following format:
- The first column is the document id (the itemid in the corresponding XML document)
- The second column is the document score (or probability).

**Describe:** 
- Python packages or modules (or any open-source software) you used
- The data structures used to represent a single document and a set of documents for each model (you can use different data structures for different models).


You also need to **test the three models on the given 50 data collections for the 50 queries (topics) by *printing out the top 15 documents* for each data collection (in descending order)**. The **output will also be put in the appendix of your final report**.

# Task 5: Model Evaluation

**Description:** Use three effectiveness measures to evaluate the three models.

In this task, you need to **use the relevance judgments (EvaluationBenchmark.zip)** to **compare with the ranking outputs in the folder of “RankingOutputs” for the selected effectiveness metric** for the three models.


You need to use the following three different effectiveness measures to evaluate the document ranking results you saved in the folder “RankingOutputs”:
1) Average precision (and MAP)
2) Precision@10 (and their average)
3) Discounted cumulative gain at rank position 10 ($p = 10$), $DCG_{10}$ (and their average):  
    $DCG_p=rel_i+\sum_{i=2}^p\frac{rel_i}{log_2(i)}$  
        $rel_i=1$ if the document at position $i$ is releveant; otherwise, it is 0.

Evaluation results can be summarized in tables or graphs. Examples are provided in the sepcification sheet.

# Task 6: Recommendation

**Description:** Recommend a model based on significance test and your analysis. 

You need to conduct a significance test to compare models. You can choose a t-test to perform a significance test on the evaluation results (e.g., in Tables 1, 2 and 3). 

You can compare models between:
- **BM25** and **JM_LM**
- **BM25** and **My_PRM**
- **JM_LM** and **My_PRM**

Based on $t$-test results ($p$-value and $t$-statistic), you can recommend a model (You ***want the proposed "My_RPM" to be the best because it is your own model***). You can perform the $t$-test using a single effectiveness measure or multiple measures. Generally, using more effectiveness measures provides stronger evidence against the null hypothesis. Note that if the $t$-test is unsatisfactory, you can use the evaluation results to refine **My_PRM** mode. For example, you can adjust parameter settings or update your design and implementation.